In [2]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/transfer/all_stages-g2/'
logger.set_logger_dir(log_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1029 13:41:43 @logger.py:94] WRN Log directory ./train_log/transfer/all_stages-g2/ exists! Please either backup/delete it, or use a new directory.
[1029 13:41:43 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1029 13:41:43 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1029 13:41:45 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-453e813e-e305-4457-bca8-d100773d6d44.json


In [3]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

#### Split train set and test set

In [4]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

#### Gether data to train rnn

In [5]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.stages = [2, 3, 4, 5, 6]
config.annotation_number = 30
dm = DataManager.from_dataset(train_set, test_set, config)
dm.get_num_info()

{'test': (4049, 17464), 'train': (3583, 16164), 'val': (829, 3651)}

In [5]:
dm.get_imbalance_ratio()

,train,val,test
embryonic dorsal epidermis,12.124542,11.560606,12.725424
procephalic ectoderm primordium,21.677215,20.815789,23.391566
embryonic head epidermis,18.472826,17.021739,19.245000
brain primordium,12.833977,12.590164,13.257042
ubiquitous,1.611516,1.467262,1.702937
embryonic ventral epidermis,13.684426,13.050847,14.573077
embryonic central nervous system,13.106299,13.050847,14.513410
dorsal prothoracic pharyngeal muscle,19.474286,19.219512,20.652406
posterior midgut primordium,8.140306,8.109890,8.416279
embryonic/larval muscle system,12.780769,13.050847,13.108014


In [6]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = False

threshold = 0.5
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config, is_finetuning=False)

In [7]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()

[1029 13:42:09 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 12
[1029 13:42:10 @input_source.py:179] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1029 13:42:10 @input_source.py:460] Setting up StagingArea for GPU prefetching ...
[1029 13:42:10 @training.py:41] Training a model of 2 towers
[1029 13:42:10 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
IN

[1029 13:42:26 @base.py:212] Creating the session ...
[1029 13:42:30 @base.py:216] Initializing the session ...
[1029 13:42:30 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1029 13:42:30 @base.py:223] Graph Finalized.
[1029 13:42:30 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1029 13:42:30 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1029 13:42:33 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1029 13:42:33 @input_source.py:419] Pre-filling staging area ...
[1029 13:42:35 @base.py:257] Start Epoch 1 ...


100%|##########|55/55[01:57<00:00, 0.42it/s]

[1029 13:44:33 @base.py:267] Epoch 1 (global_step 55) finished, time:117.86 sec.



  0%|          |0/12[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|12/12[00:10<00:00, 1.28it/s]


...

[1029 14:22:36 @saver.py:89] Model saved to ./train_log/transfer/all_stages-g2/model-1045.
[1029 14:22:39 @saver.py:158] Model with maximum 'micro_auc' saved.
[1029 14:22:39 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.669
[1029 14:22:39 @monitor.py:362] QueueInput/queue_size: 49.622
[1029 14:22:39 @monitor.py:362] coverage: 4.0469
[1029 14:22:39 @monitor.py:362] learning_rate: 1e-05
[1029 14:22:39 @monitor.py:362] loss/value: 0.034018
[1029 14:22:39 @monitor.py:362] macro_auc: 0.93857
[1029 14:22:39 @monitor.py:362] macro_f1: 0.4246
[1029 14:22:39 @monitor.py:362] mean_average_precision: 0.68398
[1029 14:22:39 @monitor.py:362] micro_auc: 0.94898
[1029 14:22:39 @monitor.py:362] micro_f1: 0.53719
[1029 14:22:39 @monitor.py:362] one_error: 0.32031
[1029 14:22:39 @monitor.py:362] ranking_loss: 0.053236
[1029 14:22:39 @monitor.py:362] ranking_mean_average_precision: 0.7508
[1029 14:22:39 @monitor.py:362] training_auc: 0.90611
[1029 14:22:39 @group.py:42] Callbacks

100%|##########|55/55[01:53<00:00, 0.49it/s]

[1029 14:24:32 @base.py:267] Epoch 20 (global_step 1100) finished, time:113.12 sec.



100%|##########|12/12[00:09<00:00, 1.19it/s]


[1029 14:24:43 @saver.py:89] Model saved to ./train_log/transfer/all_stages-g2/model-1100.
[1029 14:24:43 @monitor.py:362] DataParallelInferenceRunner/QueueInput/queue_size: 49.386
[1029 14:24:43 @monitor.py:362] QueueInput/queue_size: 49.585
[1029 14:24:43 @monitor.py:362] coverage: 4.0859
[1029 14:24:43 @monitor.py:362] learning_rate: 1e-05
[1029 14:24:43 @monitor.py:362] loss/value: 0.033164
[1029 14:24:43 @monitor.py:362] macro_auc: 0.93886
[1029 14:24:43 @monitor.py:362] macro_f1: 0.43665
[1029 14:24:43 @monitor.py:362] mean_average_precision: 0.68616
[1029 14:24:43 @monitor.py:362] micro_auc: 0.94871
[1029 14:24:43 @monitor.py:362] micro_f1: 0.55274
[1029 14:24:43 @monitor.py:362] one_error: 0.30859
[1029 14:24:43 @monitor.py:362] ranking_loss: 0.054
[1029 14:24:43 @monitor.py:362] ranking_mean_average_precision: 0.75706
[1029 14:24:43 @monitor.py:362] training_auc: 0.90969
[1029 14:24:43 @group.py:42] Callbacks took 10.444 sec in total. DataParallelInferenceRunner: 9.963sec
[102

In [7]:
test_loc_dir = 'train_log/transfer/test/D1'
test_save_name = 'max-training-auc.tfmodel'

config.proportion = {'train': 0.8, 'val': 0.2, 'test': 0.0}
config.stages = [2]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
test_date = dm.get_test_stream()
model = RNN(config, is_finetuning=True)

[1029 13:01:39 @input_source.py:147] EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue Exited.
[1029 13:01:39 @input_source.py:147] EnqueueThread QueueInput/input_queue Exited.


In [8]:
dm.get_imbalance_ratio()

,train,val,test
anlage in statu nascendi,7.439024,7.666667,7.758621
dorsal ectoderm anlage in statu nascendi,6.440860,6.090909,6.398058
maternal,3.907801,4.379310,4.013158
cellular blastoderm,1.005797,1.025974,1.005263
subset,3.552632,3.457143,3.590361
ubiquitous,0.870270,0.714286,0.809976
faint ubiquitous,5.653846,5.782609,4.953125
ventral ectoderm anlage in statu nascendi,7.650000,6.800000,7.282609
procephalic ectoderm anlage in statu nascendi,7.650000,7.666667,7.758621
posterior endoderm anlage in statu nascendi,15.878049,14.600000,15.565217


In [22]:
train_set

,,image_url,annotation
gene,stage,,
CG32656,6,"(32951_s.bmp, 32954_s.bmp, 32952_s.bmp)","(imaginal tracheal precursor, embryonic head e..."
CG3028,5,"(38372_s.bmp, 38371_s.bmp, 38374_s.bmp)","(ubiquitous, ring gland primordium)"
CG4091,6,"(9653_s.bmp, 9654_s.bmp)","(imaginal tracheal precursor, adult hindgut pr..."
CG4608,5,"(15290_s.bmp, 15298_s.bmp, 15299_s.bmp, 15289_...","(dorsal epidermis primordium, tracheal primord..."
CG3722,6,"(18010_s.bmp, 18015_s.bmp, 18020_s.bmp, 18011_...","(embryonic head epidermis, embryonic foregut, ..."
CG9184,5,"(25526_s.bmp, 25527_s.bmp, 25529_s.bmp, 25530_...","(dorsal epidermis primordium, dorsomedial neur..."
CG12653,2,"(75726_s.bmp, 75727_s.bmp, 75728_s.bmp, 75729_...","(cellular blastoderm, procephalic ectoderm anl..."
CG4123,5,"(23368_s.bmp, 23368_s.bmp, 23364_s.bmp, 23365_...","(dorsal epidermis primordium, tracheal primord..."
CG7055,5,"(17943_s.bmp, 17944_s.bmp, 17945_s.bmp, 17941_...","(embryonic central brain glia, lateral cord gl..."


In [ ]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', test_save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=log_dir + save_name, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()